In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

## REPORTE
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
ruta

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [5]:
def generar_reporte(i):
    id_reg = i
    
    # SELECCIONAR REGISTRO
    query = {'query': 'select * from dataset_318090 where id = {id_ca}'.format(id_ca=id_reg)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    if data['observaciones'] == None:
        data['observaciones']='Sin observaciones'
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    print('CAÑERO: ' + insp.canhero)
    #FOTOS
    query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_49781 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    #GENERAR DOCX
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_operador-cosechadora.docx")
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(60))})

    context = {'insp':insp, 'fotos':lista_fotos_inline}

    doc.render(context)

    # formato de nombre de archivo: "123_EVC_01-01-2022_NOMBRE"
    cod_nom = insp.canhero.split(' / ')
    file_name = ruta + '/_' + str(cod_nom[0]) + '_ECO_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(file_name + '.docx')

In [6]:
lista_id = list(range(172,173))
lista_id

[172]

In [7]:
for i in lista_id:
    print('GENERANDO id = ' + str(i))
    generar_reporte(i)
print('END=====================================')

GENERANDO id = 172
CAÑERO: 3408 / ESPINOZA TERRAZAS MIGUEL
END=====================================


## CONVERTIR A PDF

In [8]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 1 INFORMES
1 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3408_EVC_24-05-2023_ESPINOZA TERRAZAS MIGUEL_172.docx']

In [9]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3408_EVC_24-05-2023_ESPINOZA TERRAZAS MIGUEL_172.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3408_EVC_24-05-2023_ESPINOZA TERRAZAS MIGUEL_172.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3408_EVC_24-05-2023_ESPINOZA TERRAZAS MIGUEL_172.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END
